# Solve Lasso via Alternating Direction Method of Multiplier (ADMM)
This algorithm is a demo of applying ADMM solver to solve the Lasso problem. Code is not optimized, thus this algorithm is only suitable for small dimensional problems.

. $$ min \frac{1}{2}||Ax-b||_2^2 + \lambda||x||_1 $$


In [ ]:
import numpy as np
from numpy.linalg import norm
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def eta_soft(x,th):
    y = x.copy()
    for i,v in enumerate(x): 
        if v > th:
            y[i] = v - th
        elif v < -th:
            y[i] = v + th
        else:
            y[i] = 0
    return y

In [ ]:
def ADMM(A,b,lmbda, rho):
    
    
    start = time.time()
    # define algorithm parameters, iterations and tolerances
    Max_iter = 100
    Abstol = 10**-4
    Reltol = 10**-2
    
    # initialization
    m , n = A.shape
    Atb = np.matmul(A.T,b)
    AtA = np.matmul(A.T,A)
    
    x = np.zeros((n,1),dtype=np.float32)
    z = np.zeros((n,1),dtype=np.float32)
    u = np.zeros((n,1),dtype=np.float32)
    
    # iterations
    for k in range(Max_iter):
        
        # update x
        q = Atb + rho * (z - u)
        x = np.matmul(np.linalg.pinv(AtA + rho * np.eye(n,dtype=np.float32)),q)
        
        # update z
        zold = z
        z = eta_soft(x + u, lmbda / rho)
        
        # update u
        u = u + (x - z)
        
        # update residuals
        r = norm(x - z)
        s = norm(-rho * (z - zold))
        
        eps_pri = np.sqrt(n) * Abstol + Reltol * max(norm(x),norm(-z))
        eps_dual = np.sqrt(n) * Abstol + Reltol * norm(rho * u)
        
        print('r={:.5}, s={:.5}, eps_pri={:.5}, eps_dual={:.5}'.format(r,s,eps_pri,eps_dual))
        
        if (r < eps_pri) and (s < eps_dual):
            break
    
    print('Time period={} after {} iterations'.format(time.time()-start, k))
    return z
    
    

In [ ]:
np.random.seed(2)
m = 150
n = 500
sparsity = 0.02 #sparsity level
non_zero = math.floor(sparsity * n) # number of non zero values

x0 = np.random.permutation(np.concatenate((np.random.randn(non_zero,1),np.zeros((n-non_zero,1),dtype=np.float32)),axis=0))



In [ ]:
A = np.random.randn(m,n)
for i in range(n):
    A[:,i] = A[:,i]/norm(A[:,i])
    
b = np.matmul(A,x0) + np.sqrt(0.001) * np.random.randn(m,1)

lmbda_max = norm(np.matmul(A.T,b),ord=np.inf)
lmbda = 0.1 * lmbda_max
rho = 1.0
x_hat = ADMM(A,b,lmbda, rho)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(x0,label='original signal')
plt.plot(x_hat,label='estimated signal')
plt.legend()